In [1]:
import pandas as pd
import numpy as np
import scipy.io
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

### Data loading and preprocessing

In [2]:
import os

DATA_DIR = ''
if 'google.colab' not in str(get_ipython()) and "anuja" in os.environ.get('USER'):
    DATA_DIR = 'data/'
    

In [3]:
foof = pd.read_csv(DATA_DIR+"foof2features.csv")
foof = foof.rename(columns={"C1": "IDs" ,"C2": "Intercept", "C3": "Slope"})
foof

,IDs,Intercept,Slope
0,NDARAA075AMK,0.986272,1.825774
1,NDARAA112DMH,1.486650,1.888544
2,NDARAA117NEJ,1.593155,2.095749
3,NDARAA947ZG5,0.703331,1.724831
4,NDARAA948VFH,0.918020,1.749441
...,...,...,...
2037,NDARZN277NR6,1.351549,1.996940
2038,NDARZN578YDP,1.380795,2.036327
2039,NDARZN610GTY,0.339229,1.050644
2040,NDARZN677EYE,0.781225,1.470061


In [4]:
data = scipy.io.loadmat(DATA_DIR+'x.mat')  
df = pd.DataFrame(data['x'].reshape((data['x'].shape[0], -1)))
columns = np.asarray([['Electrode %d - %d/2 Hz'%(i+1, j+1)] for i in range(data['x'].shape[1]) for j in range(data['x'].shape[2])])
df.columns = columns
df['IDs'] = foof['IDs']
df



,"(Electrode 1 - 1/2 Hz,)","(Electrode 1 - 2/2 Hz,)","(Electrode 1 - 3/2 Hz,)","(Electrode 1 - 4/2 Hz,)","(Electrode 1 - 5/2 Hz,)","(Electrode 1 - 6/2 Hz,)","(Electrode 1 - 7/2 Hz,)","(Electrode 1 - 8/2 Hz,)","(Electrode 1 - 9/2 Hz,)","(Electrode 1 - 10/2 Hz,)",...,"(Electrode 105 - 71/2 Hz,)","(Electrode 105 - 72/2 Hz,)","(Electrode 105 - 73/2 Hz,)","(Electrode 105 - 74/2 Hz,)","(Electrode 105 - 75/2 Hz,)","(Electrode 105 - 76/2 Hz,)","(Electrode 105 - 77/2 Hz,)","(Electrode 105 - 78/2 Hz,)","(Electrode 105 - 79/2 Hz,)",IDs
0,7.707834e-12,1.512286e-10,2.475735e-09,3.453763e-08,4.554996e-07,0.000007,0.000109,0.001314,0.009992,0.045200,...,2.990881e-09,1.223086e-09,4.893694e-10,1.915752e-10,7.337775e-11,2.749867e-11,1.008282e-11,3.617107e-12,1.269651e-12,NDARAA075AMK
1,3.798713e-02,1.639433e-01,2.897591e-01,2.097609e-01,6.347556e-02,0.027758,0.105298,0.184513,0.130638,0.163258,...,5.835291e-06,1.856717e-06,5.922864e-07,1.928109e-07,6.449918e-08,2.204650e-08,7.598678e-09,2.604260e-09,8.780452e-10,NDARAA112DMH
2,2.104310e-06,4.299647e-05,5.644195e-04,4.760137e-03,2.579253e-02,0.089799,0.201005,0.290260,0.276177,0.196151,...,1.332064e-07,5.270957e-08,2.026971e-08,7.575282e-09,2.751336e-09,9.711398e-10,3.331297e-10,1.110552e-10,3.597966e-11,NDARAA117NEJ
3,1.692639e-01,2.021357e-01,2.273063e-01,2.406964e-01,2.400039e-01,0.225430,0.202278,0.198154,0.228356,0.181836,...,8.323825e-02,7.759521e-02,7.183415e-02,6.604061e-02,6.029416e-02,5.466678e-02,4.922161e-02,4.401211e-02,3.908162e-02,NDARAA947ZG5
4,8.332328e-06,1.552379e-04,1.804338e-03,1.287033e-02,5.615280e-02,0.149809,0.244887,0.247738,0.163687,0.094061,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NDARAA948VFH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2037,8.241315e-06,2.938160e-04,4.641777e-03,3.249546e-02,1.008094e-01,0.138615,0.084707,0.024339,0.009157,0.021386,...,5.852474e-03,4.262113e-03,3.068545e-03,2.184048e-03,1.536788e-03,1.069025e-03,7.351630e-04,4.998062e-04,3.359246e-04,NDARZN277NR6
2038,8.113400e-03,1.005675e-02,1.237930e-02,1.513279e-02,1.837070e-02,0.022147,0.026515,0.031525,0.037230,0.043721,...,5.855173e-04,3.933072e-04,2.601868e-04,1.695084e-04,1.087534e-04,6.871244e-05,4.275280e-05,2.619553e-05,1.580597e-05,NDARZN578YDP
2039,5.472767e-03,8.816325e-03,1.629356e-02,3.226822e-02,6.212373e-02,0.108594,0.166461,0.220594,0.252412,0.252807,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NDARZN610GTY
2040,2.658455e-03,5.749806e-03,1.180538e-02,2.300961e-02,4.257369e-02,0.074778,0.124685,0.197358,0.296551,0.423009,...,3.676327e-04,1.925477e-04,9.805386e-05,4.855040e-05,2.337344e-05,1.094093e-05,4.979521e-06,2.203549e-06,9.481114e-07,NDARZN677EYE


In [5]:
beh = pd.read_csv(DATA_DIR+"behaviorals.csv")
print('Before:'+str(beh.shape))

most_common_disorders = ['Attention-Deficit/Hyperactivity Disorder', 'Anxiety Disorders', 'Specific Learning Disorder',
                         'Autism Spectrum Disorder', 'Disruptive', 'No Diagnosis Given', 'Communication Disorder',
                         'Depressive Disorders']

# most_common_disorders = ['Other Neurodevelopmental Disorders', 'ADHD-Inattentive Type', 'ADHD-Combined Type', 'Anxiety Disorders', 'No Diagnosis Given', 'Depressive Disorders']

category_columns = ['DX_' + str(i).zfill(2) + '_Cat' for i in range(1, 11)] +\
                   ['DX_' + str(i).zfill(2) + '_Sub' for i in range(1, 11)]

# find users that have no diagnosis within these top diseases
# filtering should cahnge anything as this should also happen at a later stage
mask = None
for col in category_columns:
    mask_col = beh[col].isin(most_common_disorders)
    if mask is None:
        mask = mask_col
    else:
        mask = mask | mask_col

initial_size = beh.shape[0]
beh = beh[mask]
beh = beh.reset_index(drop=True)
new_size = beh.shape[0]
print('After:'+str(beh.shape))
print('Removing', initial_size - new_size,
      'patients as their diagnoses were very uncommon.')

Before:(3076, 177)
After:(2813, 177)
Removing 263 patients as their diagnoses were very uncommon.


In [6]:
no_diagnosis_given = 'No Diagnosis Given'

if no_diagnosis_given in most_common_disorders:
    no_diag_index = most_common_disorders.index(no_diagnosis_given)
    most_common_disorders = most_common_disorders[:no_diag_index] + \
        most_common_disorders[no_diag_index + 1:]

diagnoses_to_ids = {disorder: i for i, disorder in enumerate(most_common_disorders)}
diagnoses_to_ids

{'Attention-Deficit/Hyperactivity Disorder': 0,
 'Anxiety Disorders': 1,
 'Specific Learning Disorder': 2,
 'Autism Spectrum Disorder': 3,
 'Disruptive': 4,
 'Communication Disorder': 5,
 'Depressive Disorders': 6}

In [7]:
def get_disorder(data, row, index):
    disorder = data.iloc[row][category_columns[index]]

    if disorder == 'Neurodevelopmental Disorders':
        disorder = data.iloc[row][category_columns[index + 10]]

    return disorder

order_of_disorders = []
for k in range(beh.shape[0]):
    i = 0
    disorder = get_disorder(beh, k, i)
    disorders_patient = []
    while disorder != no_diagnosis_given and not pd.isnull(disorder):
        if disorder in diagnoses_to_ids:
            if diagnoses_to_ids[disorder] not in disorders_patient:
                disorders_patient.append(diagnoses_to_ids[disorder])
        i += 1
        if i == len(category_columns):
            break
        disorder = get_disorder(beh, k, i)

    order_of_disorders.append(disorders_patient)


In [8]:
max_len_order = np.max([len(x) for x in order_of_disorders])

# pad with a new token denoting the pad token
pad_token = len(most_common_disorders)
bod_token = len(most_common_disorders) + 1
eod_token = len(most_common_disorders) + 2

order_of_disorders = [[bod_token] + x + [eod_token] + [pad_token] * (max_len_order - len(x)) for x in order_of_disorders]

order_of_disorders = np.array(order_of_disorders)

classes = np.zeros((len(most_common_disorders),
                    beh.shape[0]), dtype=np.int32)

df_disorders = beh[category_columns]

for i, disorder in enumerate(most_common_disorders):
    mask = df_disorders.select_dtypes(include=[object]). \
        applymap(lambda x: disorder in x if pd.notnull(x) else False)

    disorder_df = df_disorders[mask.any(axis=1)]

    np.add.at(classes[i], disorder_df.index.values, 1)

behaviour_data_columns = beh.columns.values.astype(np.str)

columns_to_drop = behaviour_data_columns[
    np.flatnonzero(np.core.defchararray.find(behaviour_data_columns, 'DX') != -1)]

behaviour_data = beh.drop(columns=columns_to_drop)

for disorder, classification in zip(most_common_disorders, classes):
    behaviour_data[disorder] = classification

behaviour_data['order_diagnoses'] = list(order_of_disorders)


<ipython-input-8-48b714085ca7>:25: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  behaviour_data_columns = beh.columns.values.astype(np.str)


In [9]:
labels=behaviour_data[["IDs"]+list(most_common_disorders)]
labels

,IDs,Attention-Deficit/Hyperactivity Disorder,Anxiety Disorders,Specific Learning Disorder,Autism Spectrum Disorder,Disruptive,Communication Disorder,Depressive Disorders
0,NDARAA075AMK,0,0,0,0,0,0,0
1,NDARAA112DMH,1,0,0,0,1,0,0
2,NDARAA117NEJ,1,0,0,0,1,0,0
3,NDARAA306NT2,1,1,1,0,0,1,0
4,NDARAA504CRN,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...
2808,NDARZZ007YMP,0,0,0,1,0,0,0
2809,NDARZZ740MLM,1,0,0,0,0,0,0
2810,NDARZZ810LVF,0,0,0,1,0,1,0
2811,NDARZZ830JM7,0,0,0,1,0,0,0


In [10]:
df = pd.merge(df, foof, on='IDs', how='inner')
df = pd.merge(df, labels, on='IDs', how='inner')
df

,"(Electrode 1 - 1/2 Hz,)","(Electrode 1 - 2/2 Hz,)","(Electrode 1 - 3/2 Hz,)","(Electrode 1 - 4/2 Hz,)","(Electrode 1 - 5/2 Hz,)","(Electrode 1 - 6/2 Hz,)","(Electrode 1 - 7/2 Hz,)","(Electrode 1 - 8/2 Hz,)","(Electrode 1 - 9/2 Hz,)","(Electrode 1 - 10/2 Hz,)",...,IDs,Intercept,Slope,Attention-Deficit/Hyperactivity Disorder,Anxiety Disorders,Specific Learning Disorder,Autism Spectrum Disorder,Disruptive,Communication Disorder,Depressive Disorders
0,7.707834e-12,1.512286e-10,2.475735e-09,3.453763e-08,4.554996e-07,0.000007,0.000109,0.001314,0.009992,0.045200,...,NDARAA075AMK,0.986272,1.825774,0,0,0,0,0,0,0
1,3.798713e-02,1.639433e-01,2.897591e-01,2.097609e-01,6.347556e-02,0.027758,0.105298,0.184513,0.130638,0.163258,...,NDARAA112DMH,1.486650,1.888544,1,0,0,0,1,0,0
2,2.104310e-06,4.299647e-05,5.644195e-04,4.760137e-03,2.579253e-02,0.089799,0.201005,0.290260,0.276177,0.196151,...,NDARAA117NEJ,1.593155,2.095749,1,0,0,0,1,0,0
3,1.692639e-01,2.021357e-01,2.273063e-01,2.406964e-01,2.400039e-01,0.225430,0.202278,0.198154,0.228356,0.181836,...,NDARAA947ZG5,0.703331,1.724831,1,0,1,1,0,0,0
4,8.332328e-06,1.552379e-04,1.804338e-03,1.287033e-02,5.615280e-02,0.149809,0.244887,0.247738,0.163687,0.094061,...,NDARAA948VFH,0.918020,1.749441,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831,1.346991e-04,2.668833e-04,5.122547e-04,9.524861e-04,1.715693e-03,0.002994,0.005061,0.008288,0.013148,0.020205,...,NDARZM903TNL,1.423824,1.381199,0,0,0,0,0,0,0
1832,3.397339e-06,1.701445e-05,7.576350e-05,2.999317e-04,1.055570e-03,0.003303,0.009185,0.022711,0.049919,0.097542,...,NDARZN148PMN,0.168009,0.205704,0,1,0,0,0,0,0
1833,8.241315e-06,2.938160e-04,4.641777e-03,3.249546e-02,1.008094e-01,0.138615,0.084707,0.024339,0.009157,0.021386,...,NDARZN277NR6,1.351549,1.996940,1,1,0,0,0,0,1
1834,2.658455e-03,5.749806e-03,1.180538e-02,2.300961e-02,4.257369e-02,0.074778,0.124685,0.197358,0.296551,0.423009,...,NDARZN677EYE,0.781225,1.470061,1,0,0,0,0,0,0


### Data Split

In [11]:
x = df[df.columns.difference(['IDs']+most_common_disorders)]
y = df[most_common_disorders]

# summarize dataset shape
print(x.shape, y.shape)

train_features, test_features, train_labels, test_labels = train_test_split(x, y, test_size=0.3, shuffle=True)

(1836, 8297) (1836, 7)


### Metrics

In [33]:
from sklearn.metrics import hamming_loss, accuracy_score
import sklearn.metrics as skm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np


def evaluate(y_test, y_pred):
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Hamming Loss:", hamming_loss(y_test, y_pred))
    print("Classification Report:\n", skm.classification_report(y_test,y_pred, zero_division=1))
    print("Confusion matrix:\n", skm.multilabel_confusion_matrix(y_test, y_pred))

## Models

### Multi Output Classifier

In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC as svm
from sklearn.multioutput import MultiOutputClassifier

forest = RandomForestClassifier(random_state=1)
lg = LogisticRegression()
svm = svm()
models = [lg, forest, svm]

for model in models:

    multi_output_model = MultiOutputClassifier(model, n_jobs=-1)
    multi_output_model.fit(train_features, train_labels)
    predicted_labels = multi_output_model.predict(test_features)
    print(str(model)+':')
    evaluate(test_labels, predicted_labels)

LogisticRegression():
Accuracy: 0.08711433756805807
Hamming Loss: 0.27223230490018147
Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.74      0.70       349
           1       0.44      0.35      0.39       205
           2       0.26      0.23      0.24       132
           3       0.40      0.16      0.23        97
           4       0.07      0.03      0.04        92
           5       0.19      0.08      0.11        76
           6       0.23      0.11      0.14        66

   micro avg       0.48      0.39      0.43      1017
   macro avg       0.32      0.24      0.27      1017
weighted avg       0.42      0.39      0.39      1017
 samples avg       0.58      0.47      0.38      1017

Confusion matrix:
 [[[ 70 132]
  [ 89 260]]

 [[253  93]
  [133  72]]

 [[332  87]
  [102  30]]

 [[430  24]
  [ 81  16]]

 [[418  41]
  [ 89   3]]

 [[449  26]
  [ 70   6]]

 [[461  24]
  [ 59   7]]]
RandomForestClassifier(random_state=1)

### MLP

In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def get_mlp(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

n_inputs, n_outputs = x.shape[1], y.shape[1]
mlp = get_mlp(n_inputs, n_outputs)
mlp.fit(train_features, train_labels, verbose=1, epochs=100)

Epoch 1/100
41/41 [==============================] - 1s 3ms/step - loss: 0.5345
Epoch 2/100
41/41 [==============================] - 0s 2ms/step - loss: 0.4985
Epoch 3/100
41/41 [==============================] - 0s 2ms/step - loss: 0.4902
Epoch 4/100
41/41 [==============================] - 0s 2ms/step - loss: 0.4868
Epoch 5/100
41/41 [==============================] - 0s 2ms/step - loss: 0.4765
Epoch 6/100
41/41 [==============================] - 0s 2ms/step - loss: 0.4727
Epoch 7/100
41/41 [==============================] - 0s 2ms/step - loss: 0.4726
Epoch 8/100
41/41 [==============================] - 0s 3ms/step - loss: 0.4702
Epoch 9/100
41/41 [==============================] - 0s 3ms/step - loss: 0.4617
Epoch 10/100
41/41 [==============================] - 0s 2ms/step - loss: 0.4624
Epoch 11/100
41/41 [==============================] - 0s 3ms/step - loss: 0.4588
Epoch 12/100
41/41 [==============================] - 0s 3ms/step - loss: 0.4527
Epoch 13/100
41/41 [=================

In [36]:
predicted_labels_mlp = mlp.predict(test_features)
evaluate(test_labels, predicted_labels_mlp.round())

Accuracy: 0.09618874773139746
Hamming Loss: 0.2867513611615245
Classification Report:
               precision    recall  f1-score   support

           0       0.63      0.64      0.64       349
           1       0.37      0.20      0.26       205
           2       0.24      0.27      0.26       132
           3       0.21      0.16      0.19        97
           4       0.17      0.04      0.07        92
           5       0.08      0.01      0.02        76
           6       0.00      0.00      0.00        66

   micro avg       0.44      0.32      0.37      1017
   macro avg       0.24      0.19      0.21      1017
weighted avg       0.36      0.32      0.33      1017
 samples avg       0.58      0.40      0.33      1017

Confusion matrix:
 [[[ 71 131]
  [124 225]]

 [[274  72]
  [163  42]]

 [[306 113]
  [ 96  36]]

 [[395  59]
  [ 81  16]]

 [[439  20]
  [ 88   4]]

 [[463  12]
  [ 75   1]]

 [[479   6]
  [ 66   0]]]


### Binary Relevance
ignores the possible correlations between class labels

In [37]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

classifier = BinaryRelevance(GaussianNB())
classifier.fit(test_features, test_labels)

BinaryRelevance(classifier=GaussianNB(), require_dense=[True, True])

In [38]:
predicted_labels_br = classifier.predict(test_features)
evaluate(test_labels, predicted_labels_br)

Accuracy: 0.05263157894736842
Hamming Loss: 0.41405237231008557
Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.61      0.66       349
           1       0.49      0.63      0.55       205
           2       0.29      0.84      0.44       132
           3       0.37      0.36      0.36        97
           4       0.24      0.67      0.36        92
           5       0.21      0.87      0.34        76
           6       0.17      0.83      0.28        66

   micro avg       0.35      0.66      0.46      1017
   macro avg       0.36      0.69      0.43      1017
weighted avg       0.47      0.66      0.51      1017
 samples avg       0.38      0.69      0.42      1017

Confusion matrix:
 [[[119  83]
  [135 214]]

 [[211 135]
  [ 75 130]]

 [[153 266]
  [ 21 111]]

 [[394  60]
  [ 62  35]]

 [[266 193]
  [ 30  62]]

 [[228 247]
  [ 10  66]]

 [[216 269]
  [ 11  55]]]


### Classfier Chains

In [39]:
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression

classifier = ClassifierChain(LogisticRegression())
classifier.fit(test_features, test_labels)

# we should optimise this a little

/home/anujanegi/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/anujanegi/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logisti

ClassifierChain(classifier=LogisticRegression(), require_dense=[True, True])

In [40]:
predicted_labels_cc = classifier.predict(test_features)
evaluate(test_labels, predicted_labels_cc)

Accuracy: 0.9800362976406534
Hamming Loss: 0.0033704952035260565
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       349
           1       1.00      0.99      0.99       205
           2       1.00      0.98      0.99       132
           3       1.00      0.98      0.99        97
           4       1.00      0.98      0.99        92
           5       1.00      0.97      0.99        76
           6       1.00      1.00      1.00        66

   micro avg       1.00      0.99      0.99      1017
   macro avg       1.00      0.98      0.99      1017
weighted avg       1.00      0.99      0.99      1017
 samples avg       1.00      0.99      0.99      1017

Confusion matrix:
 [[[201   1]
  [  0 349]]

 [[346   0]
  [  3 202]]

 [[419   0]
  [  3 129]]

 [[454   0]
  [  2  95]]

 [[459   0]
  [  2  90]]

 [[475   0]
  [  2  74]]

 [[485   0]
  [  0  66]]]


### Label Powerset
takes correlations into account!

In [41]:
from skmultilearn.problem_transform import LabelPowerset

classifier = LabelPowerset(LogisticRegression())
classifier.fit(test_features, test_labels)

/home/anujanegi/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LabelPowerset(classifier=LogisticRegression(), require_dense=[True, True])

In [42]:
predicted_labels_lp = classifier.predict(test_features)
evaluate(test_labels, predicted_labels_lp)

Accuracy: 0.9981851179673321
Hamming Loss: 0.0010370754472387865
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       349
           1       1.00      1.00      1.00       205
           2       1.00      1.00      1.00       132
           3       1.00      1.00      1.00        97
           4       1.00      0.99      0.99        92
           5       0.99      1.00      0.99        76
           6       1.00      1.00      1.00        66

   micro avg       1.00      1.00      1.00      1017
   macro avg       1.00      1.00      1.00      1017
weighted avg       1.00      1.00      1.00      1017
 samples avg       1.00      1.00      1.00      1017

Confusion matrix:
 [[[202   0]
  [  1 348]]

 [[346   0]
  [  1 204]]

 [[419   0]
  [  0 132]]

 [[454   0]
  [  0  97]]

 [[459   0]
  [  1  91]]

 [[474   1]
  [  0  76]]

 [[485   0]
  [  0  66]]]


### Multi Label KNN

In [43]:
from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix

mlknn = MLkNN(k=10)

x_train = lil_matrix(train_features).toarray()
y_train = lil_matrix(train_labels).toarray()
x_test = lil_matrix(test_features).toarray()
# train
mlknn.fit(x_train, y_train)
# predict
# predictions_new = classifier_new.predict(x_test)
# # accuracy
# print("Accuracy = ",accuracy_score(y_test,predictions_new))
# print("\n")


/home/anujanegi/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_neighbors=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


MLkNN()

In [44]:
predicted_labels_mlknn = mlknn.predict(x_test)
evaluate(test_labels, predicted_labels_mlknn)

Accuracy: 0.13611615245009073
Hamming Loss: 0.23826808400311122
Classification Report:
               precision    recall  f1-score   support

           0       0.63      0.79      0.70       349
           1       0.44      0.24      0.31       205
           2       0.50      0.01      0.01       132
           3       0.50      0.01      0.02        97
           4       0.20      0.01      0.02        92
           5       0.00      0.00      0.00        76
           6       0.50      0.03      0.06        66

   micro avg       0.59      0.33      0.42      1017
   macro avg       0.40      0.16      0.16      1017
weighted avg       0.47      0.33      0.31      1017
 samples avg       0.67      0.42      0.38      1017

Confusion matrix:
 [[[ 42 160]
  [ 73 276]]

 [[283  63]
  [155  50]]

 [[418   1]
  [131   1]]

 [[453   1]
  [ 96   1]]

 [[455   4]
  [ 91   1]]

 [[473   2]
  [ 76   0]]

 [[483   2]
  [ 64   2]]]
